In [18]:
import numpy as np

from common.functions import sigmoid, softmax, cross_entropy_error
from common.functions import numerical_gradient

In [19]:
class TwoLayerNet:

    def __init__(self,input_size,hidden_size,output_size,
                weight_init_std=0.01):
        
        # init weight
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x):
        
        # get weight
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        # hidden layer
        # input: x * W1 + b1
        # activation: sigmoid
        # output: 
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        
        # output layer
        # input: hidden output
        # activation: softmax
        # output:
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    
    def loss(self, x, t):
        
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y==t)/float(x.shape[0])
        
        return accuracy
    
    def numerical_gradient(self, x, t):
        
        loss_W = lambda W: self.loss(x, t)
        
        grades = {}
        grades['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grades['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grades['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grades['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grades
        

In [20]:
from common.datasets import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(one_hot_label=True)

In [21]:
print(x_train.shape)
print(x_test.shape)

(60000, 784)
(10000, 784)


In [27]:
train_loss_list = []
train_acc_list = []
test_acc_list = []

iters_num = 1000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

iter_per_epoch = max(train_size / batch_size, 1)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    
    # mini_batch
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # numerical gradient for mini_batch
    grad = network.numerical_gradient(x_batch, t_batch)
    
    # update weight
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] = learning_rate * grad[key]
    
    # caculate the loss
    loss = network.loss(x_batch, t_batch)

    # record train step
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        
        # model accuracy for x_train
        train_acc = network.accuracy(x_train, t_train)
        # model accuracy for x_test
        test_acc = network.accuracy(x_test, t_test)
        
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)

        print('epoch =', i)
        print('train acc, test acc |', str(train_acc), ',' + str(test_acc))


epoch = 0
train acc, test acc | 0.10441666666666667 ,0.1028


KeyboardInterrupt: 